In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Set the paths for your training, validation, and test datasets
train_dir = '/content/drive/MyDrive/data/sdss_dataset_split/Train'
val_dir = '/content/drive/MyDrive/data/sdss_dataset_split/Validation'
test_dir = '/content/drive/MyDrive/data/sdss_dataset_split/Test'

# Define image size and batch size
img_height, img_width = 224, 224
batch_size = 32

# Load the ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Unfreeze the last few layers for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Added dropout to prevent overfitting
x = Dense(3, activation='softmax')(x)  # 3 classes: 5, 6, 7


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation and data generators for loading images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,  # Added vertical flip
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Learning rate reduction on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Checkpoint to save the best model
checkpoint_filepath = '/content/drive/MyDrive/best_resnet_model.keras'
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

Found 5240 images belonging to 3 classes.
Found 650 images belonging to 3 classes.
Found 650 images belonging to 3 classes.


In [ ]:
history = model.fit(
    train_generator,
    epochs=30,  # Increased number of epochs
    validation_data=val_generator,
    callbacks=[reduce_lr, model_checkpoint]
)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.4668 - loss: 1.6120 
Epoch 1: val_accuracy improved from -inf to 0.40462, saving model to /content/drive/MyDrive/best_resnet_model.keras
164/164 ━━━━━━━━━━━━━━━━━━━━ 3161s 18s/step - accuracy: 0.4673 - loss: 1.6092 - val_accuracy: 0.4046 - val_loss: 3.1433 - learning_rate: 1.0000e-04
Epoch 2/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step - accuracy: 0.6757 - loss: 0.7569
Epoch 2: val_accuracy improved from 0.40462 to 0.40615, saving model to /content/drive/MyDrive/best_resnet_model.keras
164/164 ━━━━━━━━━━━━━━━━━━━━ 76s 445ms/step - accuracy: 0.6758 - loss: 0.7568 - val_accuracy: 0.4062 - val_loss: 2.1750 - learning_rate: 1.0000e-04
Epoch 3/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.7540 - loss: 0.6385
Epoch 3: val_accuracy did not improve from 0.40615
164/164 ━━━━━━━━━━━━━━━━━━━━ 69s 402ms/step - accuracy: 0.7540 - loss: 0.6385 - val_accuracy: 0.2815 - val_loss: 1.9977 - learning_rate: 1.0000e-04
Epoch 4/30
164/164

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 401s 20s/step - accuracy: 0.8842 - loss: 0.3059
Test accuracy: 0.8769230842590332


In [ ]:
# Predicting on new data
predictions = model.predict(test_generator)


21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 257ms/step


In [ ]:
# Save the final model with the .keras extension
final_model_filepath = '/content/drive/MyDrive/final_resnet_model.keras'
model.save(final_model_filepath)

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/final_resnet_model.keras')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Define image size expected by the model
img_height, img_width = 224, 224

# Load and preprocess the image
img_path = '/content/drive/MyDrive/data/sdss_dataset_split/Test/6/image_0_class_6.jpg'
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize pixel values to [0, 1]


In [ ]:
# Make prediction
predictions = model.predict(img_array)

# Get the predicted class
predicted_class = np.argmax(predictions, axis=1)
print(f'Predicted class index: {predicted_class[0]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted class index: 1


In [ ]:
# Define class labels
class_labels = ['Class 5', 'Class 6', 'Class 7']

# Map the index to label
predicted_label = class_labels[predicted_class[0]]
print(f'Predicted class label: {predicted_label}')


Predicted class label: Class 6
